In [1]:
import openai

f = open('../api_key.txt')
api_key = f.read()
openai.api_key = api_key

In [ ]:
model = 'gpt-3.5-turbo'
messages=[
    {'role':'system', 'content':'You are a helpful assistant'},
    {'role':'user', 'content':'hello'}
]

completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
)

In [2]:
import json

# 날씨를 물어보면 같은 답을 return하는 dummy function 생성.
def get_current_weather(location, unit='fahrenheight'):
    weather_info = {
        'location': location,
        'temperature': '72',
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }
    return json.dumps(weather_info)

messages = [{'role': 'user', 'content': "What's the weather like in Boston?"}]
functions = [
    {
        'name': 'get_current_weather', # function의 이름
        'description': 'Get the current weather in a given location', # 해당 function의 설명
        'parameters': { 
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The city and state, e.g. San Francisco, CA' # 파라미터의 속성 설명
                },
                'unit': {
                    'type': 'string',
                    'enum': ['celsius', 'fahrenheit']
                }
            },
            'required': ['location']
        }
    }
]

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions,
    function_call='auto'
)
response_message = response['choices'][0]['message']
print(response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston\"\n}"
  }
}


In [3]:
if response_message.get('function_call'):
    available_fuctions = {
        'get_current_weather': get_current_weather
    }
    function_name = response_message['function_call']['name']
    function_to_call = available_fuctions[function_name]
    function_args = json.loads(response_message['function_call']['arguments'])
    function_response = function_to_call(
        location=function_args.get('location'),
        unit=function_args.get('unit')
    )
    
    print(function_response)


{"location": "Boston", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}


In [4]:
messages.append(response_message)
messages.append(
    {
        'role': 'function',
        'name': function_name,
        'content': function_response
    }
)

print(messages)

[{'role': 'user', 'content': "What's the weather like in Boston?"}, <OpenAIObject at 0x2a9f60bfb80> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston\"\n}"
  }
}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}'}]


In [7]:
second_response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo-0613',
    messages=messages
)
print(second_response)

{
  "id": "chatcmpl-7kq0hKdOIiNlBi4GZjh2GZhkUiflE",
  "object": "chat.completion",
  "created": 1691397371,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 17,
    "total_tokens": 85
  }
}


In [8]:
print(second_response.choices[0]['message']['content'])

The weather in Boston is currently sunny and windy with a temperature of 72 degrees.
